
# Real-time Pose Detection for Car Control

This notebook demonstrates how to use real-time pose detection from a camera stream to control a remote car's movements. Utilizing OpenCV and a custom pose detection module, we can interpret specific poses as commands to move the car in different directions.
    

In [ ]:

import cv2
import numpy as np
import socket
import json
import time
import requests
from utils.PoseDetectorModule import PoseDetector
    


## Initializing the Pose Detector

We start by initializing the pose detector, specifying that we're interested in detecting upper body poses.
    

In [ ]:

detector = PoseDetector(upBody=True)
    


## Setting Up Streaming and Car Control

Next, we set up the stream URL for the car's camera and define the car's IP address and port for sending control commands.
    

In [ ]:

stream_url = "http://192.168.4.1:81/stream"
IP = "192.168.4.1"
PORT = 100
    


## Function to Send Commands to the Car

This function simplifies sending commands to the car by wrapping the socket connection logic.
    

In [ ]:

def send_command(command):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((IP, PORT))
        sock.sendall(json.dumps(command).encode())
        time.sleep(0.5)
    


## Processing the Camera Stream for Pose Detection

We access the camera stream using a `requests` session. Each frame is processed to detect poses, and specific poses trigger car movement commands.
    

In [ ]:

# Start streaming session
session = requests.Session()
response = session.get(stream_url, stream=True)

if response.status_code == 200:
    bytes_data = bytes()
    while True:
        chunk = response.raw.read(1024)
        if not chunk:
            break
        bytes_data += chunk
        a = bytes_data.find(b'ÿØ')  # JPEG start
        b = bytes_data.find(b'ÿÙ')  # JPEG end
        while a != -1 and b != -1:
            jpg = bytes_data[a:b+2]
            bytes_data = bytes_data[b+2:]
            img = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
            
            if img is not None:
                img = cv2.flip(img, 1)
                img = detector.findPose(img, draw=True)
                lmList, bboxInfo = detector.findPosition(img, draw=True)

                # Processing and sending commands based on detected poses
                # Add your pose detection logic here

                cv2.imshow('Elegoo Smart Car Camera Stream', img)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            a = bytes_data.find(b'ÿØ')
            b = bytes_data.find(b'ÿÙ')

cv2.destroyAllWindows()
    


## Exercises for Students

1. **Enhance Pose Detection**: Experiment with detecting different poses and associating them with unique car movements or actions.

2. **Integrate Additional Commands**: Based on the detected poses, create more complex sequences of commands for the car to execute.

3. **Optimize Command Timing**: Fine-tune the sleep durations and command parameters to achieve smoother car control.

4. **User Interface for Monitoring**: Develop a simple UI to display the detected poses and the commands being sent to the car in real-time.
    